# Open Beauty Facts
### 1. Enunciado
Para el desarrollo de esta actividad es necesario descargar el dataset de "Open Beauty Facts" en formato CSV
Una vez descargado el fichero, vamos a subirlo a Google Cloud y almacenarlo en un bucket de Google Storage:

- Entrar en Google Cloud, ir a Cloud Storage, y crear un bucket con el nombre "openfoodfacts_expediente", donde "expediente" es tu número de expediente.
- Subir al bucket que has creado el CSV con el dataset descargado previamente.
- Crear otro bucket con el nombre "output_expediente" que es donde guardaremos los datos de salida.
A continuación vamos a examinar el dataset y pensar en conjuntos de datos e información que se puedan extraer del dataset en bruto usando pySpark.

Para la ejecución del programa pySpark (puede ser un programa python (.py) o un Jupyter Notebook (.ipynb) crear un cluster en Google cloud usando DataProc y ejecutar el código en el cluster. Guardar los datos de salida en Google Storage en el bucket que creamos para la salida.

Por último vamos a crear una visualización de datos usando la herramienta Google Looker Studio o una visualización creada ad-hoc con librerías de gráficos para mostrar los datos y la información extraída con Spark.

### 2. Librerías

In [2]:
import findspark
findspark.init()

import pyspark 
from pyspark.sql import SparkSession
from datetime import datetime
from pyspark.sql.functions import isnan, when, count, col

spark = SparkSession.builder.getOrCreate()

### 3. Carga de datos y limpieza

In [1]:
obf = spark.read.csv('gs://openbeatyfacts_22076186/obf.csv', sep = '\t', header = True, inferSchema = 'true')
obf.show(10)

In [3]:
#obf = spark.read.csv('./obf.csv', sep = '\t', header = True, inferSchema = 'true')
#obf.show(10)


+---------+--------------------+--------------------+----------+-------------------+---------------+----------------------+--------------------+--------------------+--------+--------------------+--------------------+---------------------+------------------------+-----------------+-----------------+--------------------+--------------------+--------------------+-------+------------+--------------------+-------------------------+------------------+-----------------+--------------+---------+--------------+------------------------+------+-----------+---------------+------+----------------+--------------------+--------------+--------------------+---------+------------+------+-----------+---------+------------+----------------+-----------+---------+--------------------+--------------------+---------------------------+-------------------------+------------------------------+---------------------------------------+-------------------------------------+------------------------------------------+

In [12]:
# tamaño del dataset
print("El dataset tiene", obf.count(),"filas y", len(obf.columns),"columnas")

El dataset tiene 31512 filas y 176 columnas


In [14]:
# contar el total de filas
total_rows = obf.count()
# calculamos el número de valores nulos en cada columna
null_counts = obf.select([count(when(col(c).isNull(), c)).alias(c) for c in obf.columns])
# calculamos el porcentaje de valores nulos en cada columna
null_percentages = null_counts.select([(col(c) / total_rows * 100).alias(c) for c in null_counts.columns])
# mostramos el resultado
null_percentages.show()

+----+---+------------------+---------+----------------+---------------+----------------------+------------------+-----------------+------------------+-----------------+-----------------+---------------------+------------------------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+--------------------+-------------------------+-----------------+-----------------+-----------------+-----------------+-----------------+------------------------+------+-----------------+-----------------+-----------------+------------------+------------------+------------------+-----------------+----------------+------------+----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+---------------------------+-------------------------+------------------------------+---------------------------------------+-------------------------

In [16]:
# borramos las filan en las cual el nombre del producto es nulo
obf = obf.filter(obf.product_name.isNotNull())
obf.count()

24322

In [17]:
# borramos las filas que puedan estar duplicadas
obf = obf.dropDuplicates()
obf.count()

23386

In [22]:
# nos quedamos con code, product_name, brands, categories_en y countries_en porque son las columnas que nos interesan y las que mas datos tienen
obf2 = obf.select('code', 'product_name', 'brands', 'categories_en', 'countries_en')
obf2.show(10)

+------------+--------------------+------------+--------------+--------------------+-------------+
|        code|        product_name|    quantity|        brands|       categories_en| countries_en|
+------------+--------------------+------------+--------------+--------------------+-------------+
|       37541| Ultimate Mouth Wash|        null|          null|                null|United States|
| 19100110656|Ultra Healing Ext...|  26.5 FL OZ|       Jergens|                null|United States|
| 22796924179|Aegean oil of Mor...|19.5 Fl. Oz.|     Body wash|                null|United States|
| 37000878100| Savon Dawn Platinum|        null|          null|                null|       Canada|
|   405518653|La petite robe no...|      100 ml|Guerlain Paris|                null|       France|
|     4254205|        Nord Extrême|       50 ml|   Narta Homme|Hygiene,Deodorant...|       France|
|682223020197|Jack Black Double...|        null|    jack black|                null|United States|
|731064602

In [23]:
# contar las apariciones de cada pais
obf2.groupBy('countries_en').count().orderBy('count', ascending=False).show(10)

+--------------+-----+
|  countries_en|count|
+--------------+-----+
|        France|13840|
|       Germany| 1609|
|         Spain| 1162|
| United States|  707|
|       Belgium|  326|
|   Switzerland|  316|
|         Italy|  295|
|          null|  291|
|        Canada|  289|
|United Kingdom|  287|
+--------------+-----+
only showing top 10 rows



In [24]:
# contar las apariciones de cada marca
obf2.groupBy('brands').count().orderBy('count', ascending=False).show(10)

+-----------+-----+
|     brands|count|
+-----------+-----+
|       null| 3700|
|        Axe|  469|
|     Signal|  414|
|      Nivea|  386|
|       Dove|  358|
|Yves Rocher|  272|
|  Carrefour|  227|
|    Garnier|  197|
|   Monsavon|  182|
|    Timotei|  155|
+-----------+-----+
only showing top 10 rows



In [25]:
# contar las apariciones de cada categoria
obf2.groupBy('categories_en').count().orderBy('count', ascending=False).show(10)

+--------------------+-----+
|       categories_en|count|
+--------------------+-----+
|                null|13458|
|Non food products...|  560|
|       Hair,Shampoos|  458|
|Hygiene,Showers-a...|  425|
|  Hygiene,Deodorants|  415|
| Hygiene,Toothpastes|  354|
|       Hygiene,Soaps|  268|
|Non food products...|  239|
|    Body,Hand-creams|  167|
|Suncare,In-sun-pr...|  144|
+--------------------+-----+
only showing top 10 rows



In [28]:
# guardar el dataset en un csv
obf.toPandas().to_csv("./obbflimpio.csv", header=True)

TypeError: Casting to unit-less dtype 'datetime64' is not supported. Pass e.g. 'datetime64[ns]' instead.